This notebook does the same thing as 'DeformVolume.py' but is also a step-by-step guide through the process.

Firstly we define the modules that we are going to be using:

In [ ]:
#External Modules
import re
import shutil
import argparse
import numpy as np
import SimpleITK as sitk
from pathlib import Path

#Platipy Modules
from platipy.imaging.registration.utils import apply_transform

#Custom Mdules
from DeformHeadCT.VolumeInfo import VolumeDeformation
from DeformHeadCT.deformation import (
    PrepareRegistration,
    HeadDeformation
    )

Next we will define some input parameters, including the location of the dicom files to be processed, and what kind of deformation will be applied to the volumes. 
Shown below is both options inlcuding how to use a json file to initialise the functions, and how to manually enter the parameters:

In [ ]:
#This is the file name of the elastix parameter file used to do the rigid/non-rigid registration. An example parameter file is found in the "examples" directory
RegParamFile = 'examples/Elastix_BSpline_OpenCL_RigidPenalty.txt'

How to Initialise using a json file:

In [ ]:
#This is the file name of the json file that contains the volume and deformation parameters 
InfoFile = 'examples/OneAxisRotationANDGTVShift.json'
#Get volume information from json file
VolInfo = VolumeDeformation(InfoFile=InfoFile)

How to Initialise using manual parameters (uncomment the code below to run):

In [ ]:
#name = ""
#InputDirectory = ""
#StructureFile = ""
#TempDirectory = examples/TestingTempDir
#OutputDirectory = examples/TestingGitHubThing
#axes = [-1, 0, 0] 
#angles = [2.5] 
#coordinates_cutoff = [[67, 268, 254], [58, 196, 254]]
#VertDict = {}
#VertDict["Oc-C1"] = [100, 229, 258] 
#VertDict["C1-C2"] = [93, 228, 258] 
#VertDict["C2-C3"] = [84, 227, 257] 
#Structure_Names="GTV_66",
#Structure_Shift = [3.8,2.2,0]

#VolInfo = VolumeDeformation(patient_id = name,axes = axes,angles = angles,InputDir = '',
#                            StructDir = StructureFile, OutputDir = OutputDirectory,
#                            nifti_directory = TempDirectory, Structure_Names = Structure_Names,
#                            coordinates_cutoff = coordinates_cutoff, VertDict = VertDict,
#                            Structure_Shift = Structure_Shift)

Organise the dcm files and then convert the files from dicom to nifit format

In [ ]:
VolInfo.PrepareDcmData()

Initialise the head deformation class, which contains the calculated dvf to deform the initial ct to the final deformed ct. 

In [ ]:
HeadDef = HeadDeformation(VolInfo.nifti_directory,VolInfo.patientunderscore,structShiftFlag=1)

Apply the head rotation as defined by the angles and axes parameters

In [ ]:
HeadDef.ApplyHeadRotation(VolInfo.coordinates_cutoff,VolInfo.angles,VolInfo.axes,VolInfo.point_of_rotation) 

Apply the rigid shift of the structures as defined by the Structure_Names and Structure_Shift parameters

In [ ]:
HeadDef.ApplyGTVShift(VolInfo.Structure_Shift,VolInfo.Structure_Names)

Convert the calculated DVF to a SimpleITK transform type and deform the initial ct using the unsmoothed dvf

In [ ]:
InitialTransform = sitk.DisplacementFieldTransform(
    sitk.Cast(HeadDef.dvf_field, sitk.sitkVectorFloat64)
)
#Deform CT volume using initial transformation
image_ct_deformed = apply_transform(HeadDef.image_ct, transform=InitialTransform, interpolator=sitk.sitkLinear)

Setup and apply the rigid/non-rigid registration completed using the elastix program

In [ ]:
dvf_New = PrepareRegistration(HeadDef.image_ct,image_ct_deformed,VolInfo.nifti_directory,RegParamFile)

Apply dvf to CT image and structures

In [ ]:
#Cast dvf prior to transformation
FinalTransform = sitk.DisplacementFieldTransform(
    sitk.Cast(dvf_New, sitk.sitkVectorFloat64)
)

#Use DVF to deform CT
image_ct_deformed2 = apply_transform(HeadDef.image_ct, transform=FinalTransform, interpolator=sitk.sitkLinear)

deformed_structures = {}

#apply dvf field to deformed_structures   
structures = {re.findall(".*_RTSTRUCT_(.*).nii.gz", p.name)[0]: sitk.ReadImage(str(p)) for p in HeadDef.structure_paths}
for struct in structures:
    deformed_structures[struct] = apply_transform(structures[struct], transform=FinalTransform, default_value=0,interpolator=sitk.sitkLinear)

Write the deformed ct volume and structures in a dicom format

In [ ]:
VolInfo.WriteVolumesToFile(image_ct_deformed2,1,deformed_structures)

(Optional) Delete the temporary files that were created to save space

In [ ]:
shutil.rmtree(input_dcm_dir = str(VolInfo.nifti_directory) + '/dicom')
shutil.rmtree(VolInfo.nifti_directory)
if structShiftFlag:
    shutil.rmtree(Path(VolInfo.OutputDir + '/OutputStructures'))